In [6]:
import yaml
import os
import pathlib
import time
from Agent.xwAPI import xwBackend
from utils.construct_prompt import get_api_doc
from utils.ChatGPT import ChatGPT

In [7]:
import evaluate

In [8]:
CONFIG_FILE = "./config/config.yaml"

In [4]:
def set_agent_configuration(configuration_file_path):
    with open(configuration_file_path, mode='r') as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]
    agent_config['ChatGPT_1']['api_keys'] = [os.environ['OPENAI_API_KEY']]
    return agent_config

In [5]:
def get_api_documentation(agent_config):
    with open(agent_config['api_doc_path']) as f:
        api_doc = yaml.load(f, Loader=yaml.FullLoader)
    return api_doc

In [6]:
def get_excel_backend(agent_config, api_doc):
    if agent_config['API_backend'] == 'xw':
        xw_backend = xwBackend(agent_config['APP_backend'], api_doc)
    return xw_backend

In [7]:
def create_path_generator(file_path):
    return pathlib.Path(f"{file_path}").glob("**/*")

In [8]:
def get_source_excel_filenames(path_generator):
    response_log_paths = [str(path).split("\\")[-1] for path in path_generator if path.is_file()]
    return [f"{log_file[2:-5]}.xlsx" for log_file in response_log_paths]

In [9]:
agent_config = set_agent_configuration(configuration_file_path=CONFIG_FILE)
api_doc = get_api_documentation(agent_config=agent_config)
xw_backend = get_excel_backend(agent_config=agent_config, api_doc=api_doc)

In [11]:
path_generator = create_path_generator(file_path="../output_dir/refined_responses/")
input_file_paths = [str(path) for path in path_generator]
input_file_paths

['..\\output_dir\\refined_responses\\1_Dragging.yaml',
 '..\\output_dir\\refined_responses\\2_Dragging.yaml',
 '..\\output_dir\\refined_responses\\2_FutureValue.yaml',
 '..\\output_dir\\refined_responses\\4_Dragging.yaml',
 '..\\output_dir\\refined_responses\\4_FutureValue.yaml',
 '..\\output_dir\\refined_responses\\5_BoomerangSales.yaml']

In [14]:
path_generator = create_path_generator(file_path="../output_dir/refined_responses/")
excel_file_names = get_source_excel_filenames(path_generator=path_generator)
excel_file_paths = [f"../dataset/task_sheets/{file_name}" for file_name in excel_file_names]
excel_file_paths

['../dataset/task_sheets/Dragging.xlsx',
 '../dataset/task_sheets/Dragging.xlsx',
 '../dataset/task_sheets/FutureValue.xlsx',
 '../dataset/task_sheets/Dragging.xlsx',
 '../dataset/task_sheets/FutureValue.xlsx',
 '../dataset/task_sheets/BoomerangSales.xlsx']

In [16]:
example_index = 1
example_file_path = excel_file_paths[example_index]
example_input_file_path = input_file_paths[example_index]
example_file_path, example_input_file_path

('../dataset/task_sheets/Dragging.xlsx',
 '..\\output_dir\\refined_responses\\2_Dragging.yaml')

In [17]:
path_generator = create_path_generator(file_path="../output_dir/intermediate_responses/")
correct_response_file_paths = [str(path) for path in path_generator]

In [18]:
test_index = 2
test_file_path = correct_response_file_paths[test_index]
test_file_path

'..\\output_dir\\intermediate_responses\\2_FutureValue.yaml'

In [19]:
def get_sheet_state(file_path):
    if file_path is not None:
        time.sleep(0.5)
        xw_backend.OpenWorkbook(file_path)

    return xw_backend.GetSheetsState()

In [20]:
example_sheet_state = get_sheet_state(file_path=example_file_path)
example_sheet_state

'Sheet state: Sheet "Sheet1" has 2 columns (Headers are A: "Hanging Mass (m2) (kg)", B: "Acceleration (m/s^2)") and 122 rows (1 header row and 121 data rows).'

In [ ]:
prompt_format = agent_config['ChatGPT_1']['prompt_format']
api_list, api_usage, api_detail_doc = get_api_doc(prompt_format, api_doc)

In [ ]:
response_log_path = f"../output_dir/refined_responses/{response_log_paths[example_index]}"
with open(response_log_path) as file:
    response_log = yaml.load(file, Loader=yaml.Loader)

In [ ]:
intermediate_response_log_path = intermediate_response_paths[example_index]
with open(intermediate_response_log_path) as file:
    intermediate_response_log = yaml.load(file, Loader=yaml.Loader)

In [ ]:
prompt = """
SYSTEM
Summarize the each sub-step of refined responses into explanations in natural language.

USER
{one_shot_refined_response_example}
Here is the supplementary documentation you can reference:
{one_shot_documentation_example}
Here is the corresponding sheet state:
{one_shot_sheet_state_example}

ASSISTANT
{one_shot_intermediate_response_example}

USER
{actual_refined_response}
Here is the supplementary documentation you can reference:
{actual_documentation}
Here is the corresponding sheet state:
{actual_sheet_state}
"""
prompt

In [ ]:
# Filter upto first open paranthesis
unique_function_names = set([sub_instruction[:sub_instruction.find("(")] for sub_instruction in response_log["refined_response"]])
unique_function_names

In [ ]:
documentation_for_functions = {"documentation": [api_detail_doc[name] for name in unique_function_names]}

In [ ]:
prompt = prompt.format(one_shot_refined_response_example=yaml.dump(response_log["refined_response"]),
                       one_shot_documentatio_examplen=yaml.dump(documentation_for_functions["documentation"]), 
                       one_shot_sheet_state_example=sheet_state["Sheet state"])
print(prompt)

In [ ]:
async def call_chat_gpt():
    try:
        chatbot = ChatGPT(agent_config['ChatGPT_1'], context=[], interaction_mode=True)
        response = await chatbot(prompt)
    except Exception as e:
        print(f"error occurs when parsing response: {e}")
    else:
        return response


In [ ]:
response = await call_chat_gpt()

In [ ]:
print(response)